In [1]:
%reload_ext autoreload
%autoreload 2

from __future__ import (absolute_import, division, print_function, unicode_literals)
import backtrader as bt
import pandas as pd
from datetime import datetime
from colorama import Fore, Back, Style
import sys
from pprint import pprint
from IPython.display import clear_output
import IPython
from IPython.display import display_html

import matplotlib 
matplotlib.rc("font", family='Microsoft YaHei')# 增加


sys.path.append("..")
from TestStrategy import TestStrategy
# from NoStrategy import NoStrategy




cerebro = bt.Cerebro()

# 增加一个策略
cerebro.addstrategy(TestStrategy )

#获取数据
stock_hfq_df = pd.read_csv("./sz000001.csv",index_col='date',parse_dates=True)
start_date = datetime(2019, 8, 1)  # 回测开始时间
end_date = datetime(2022, 8, 30)  # 回测结束时间
data = bt.feeds.PandasData(dataname=stock_hfq_df, fromdate=start_date, todate=end_date)  # 加载数据
cerebro.adddata(data)  # 将数据传入回测系统


cerebro.broker.setcash(100)

cerebro.run()
%matplotlib widget 
cerebro.plot( style='bar',iplot=False)




TestStrategy __init   &&&&&
Bias __init__
good day
>>>>>>>>>>
当前可用资金 86.23
当前总资产 99.97
当前持仓量 1
当前持仓成本 13.77
<<<<<<<<<<<<<



[[<Figure size 1280x960 with 5 Axes>]]